In [1]:
%pip install tqdm langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone docx2txt

  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/46.3 kB ? eta -:--:--
   ---------------------------------------- 46.3/46.3 kB ? eta 0:00:00
   ---------------------------------------- 0.0/516.3 kB ? eta -:--:--
   ---------------------------------------- 516.3/516.3 kB ? eta 0:00:00
   ---------------------------------------- 0.0/239.1 kB ? eta -:--:--
   ---------------------------------------- 239.1/239.1 kB ? eta 0:00:00
   ---------------------------------------- 0.0/363.8 kB ? eta -:--:--
   ---------------------------------------- 363.8/363.8 kB ? eta 0:00:00
   ---------------------------------------- 0.0/52.2 kB ? eta -:--:--
   ---------------------------------------- 52.2/52.2 kB ? eta 0:00:00
   ---------------------------------------- 0.0/42.3 kB ? eta -:--:--
   ---------------------------------------- 42.3/42.3 kB ? eta 0:00:00
   ---------------------------------------- 0.0/44.3 kB ? eta -:--:--
   ----


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# 청킹 작업
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)


225

In [7]:
document_list[52]

Document(metadata={'source': './tax.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 20

In [8]:
# 임베딩
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-large")# 3072

In [28]:
# pinecone에 데이터 로딩
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = "tax-index"
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

vectorstore = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [9]:
# import os
# from pinecone import Pinecone
# from langchain_pinecone import PineconeVectorStore

# index_name = "tax-index"
# pinecone_api_key = os.getenv("PINECONE_API_KEY")
# pc = Pinecone(api_key=pinecone_api_key)
# pinecone_index = pc.Index(index_name)

# vectorstore = PineconeVectorStore(
#     index=pinecone_index,
#     embedding=embedding,
#     text_key="text"
# )

# texts = [doc.page_content for doc in document_list]
# metadatas = [doc.metadata for doc in document_list]

# batch_size = 2

# for i in range(0, len(texts), batch_size):
#     batch_texts = texts[i:i+batch_size]
#     batch_metadatas = metadatas[i:i+batch_size]

#     try:
#         vectorstore.add_texts(texts=batch_texts, metadatas=batch_metadatas)
#     except Exception as e:
#         print(f"Error adding batch {i // batch_size + 1}: {e}")



d:\AI_Prompt\workspace\ai_agent_work2\tax_env\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
# query = "연봉 5천만원인 거주자의 종합소득세는?"
query = "연봉 5천만원 직장인의 소득세는?"

In [30]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
retriever.invoke(query)

[Document(id='cc07eb11-54c3-4b71-9f38-97d4c43c2f98', metadata={'source': './tax.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 제21조제1항제18호 및 제21호에 

In [31]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

d:\AI_Prompt\workspace\ai_agent_work2\tax_env\myenv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [32]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [33]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
)

In [34]:
# query = "연봉 5천만원인 거주자의 종합소득세는?"
query = "연봉 5천만원 직장인의 소득세는?"

In [35]:
ai_message = qa_chain.invoke({"query": query})

In [36]:
ai_message

{'query': '연봉 5천만원 직장인의 소득세는?',
 'result': '연봉 5천만원 직장인의 소득세는 기본세율에 따라 부과됩니다.  자세한 세율은 개인의 소득 구간과 공제 항목에 따라 달라질 수 있기 때문에, 정확한 세액을 계산하기 위해서는 소득세법상 각 구간에 따른 세율 표를 참고해야 합니다.'}

In [37]:
query = "연봉 5천만원인 거주자의 종합소득세는?"
ai_message = qa_chain.invoke({"query": query})
ai_message

{'query': '연봉 5천만원인 거주자의 종합소득세는?',
 'result': '연봉이 5천만원인 거주자의 종합소득세는 624만원입니다. 이는 84만원과 1,400만원을 초과하는 나머지 3,600만원에 대한 15%를 더하여 계산된 금액입니다. 따라서, 84만원에 3,600만원의 15%에 해당하는 540만원을 더해 624만원이 됩니다.'}

In [45]:
# dictionary prompt
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사저을 참고해서 사용자의 질문을 변경해 주세요.
    만약 변경할 필요가 없다고 판단 된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요.
    사전: {dictionary}
    질문: {{question}}
    """)

dictionary_chain = prompt | llm | StrOutputParser()
tax_chain = {"query": dictionary_chain} | qa_chain

In [47]:
new_question = dictionary_chain.invoke({"question": query})
new_question

'연봉 5천만원인 거주자의 종합소득세는?'

In [50]:
ai_message = tax_chain.invoke({"question": query})
ai_message

{'query': '연봉 5천만원인 거주자의 종합소득세는?',
 'result': '연봉 5천만 원인 거주자의 종합소득세는 624만 원입니다. 이는 과세표준 5천만 원 이하 구간에 해당하므로, 84만 원 및 1,400만 원을 초과하는 금액의 15%를 더한 금액으로 계산됩니다: \\( 84만 + (5천만 - 1,400만) \\times 0.15 = 624만 \\) 원입니다.'}